#### Durability
The purpose of a database system is to proide a safe place where data can be stored without fear of losing it. DUrability is the promise that once a transaction has committed successfully, any data it has written will not be forgotten, even if there is a hardware fault or the database crashes.

In a single-node databse, durability typically means that the data has been written to nonvolatile storage as a hard dirve or SSD. It usually also invlodes a write-ahead log or similar, which allows recovery in the event that the data structures on disk are corrupted. In a replicated database, durability may mean that the data has been successfully copied to some number of nodes. In order to provide a durability guarantee, a database mush wait until these writes or replications are complete before reporting a transaction as successfully committed. 

As discussed in Reliability on page 6, perfect durability does not exists: if all your hard disks and all your backups are destroyed at the same time, there's obviously onthing your databse can do to save you.

Some databses also provide more complex atomic operations, such as an increment operation, which removes the need for a read-modify-write cycle like that in Figure 7-1. Similarly popular is a compare-and-set operation, which allows a write to happen only if the value has not been concurrently changed by someone else. 

These single-object operations are useful, as they can prevent lost updates when several clients try to write to the same object concurrently. However, they are not transactions in the usual sense of the word. Compare-and-set and other single-object operations have been dubbed "light-weight transactions" or even "ACID" for marketing purposes, but the terminology is misleading. A transaction is usually understood as a mechanism for grouping multiple operations on multiple objects into one unit of execution.

#### The need for multi-object transactions

Many distributed datastores have abandoned multi-object transactions because they are difficult to implement across partitions, and they can get in the way in some scenarios where very high availabilty or performance is required. Howeer, there is nothing that fundamentally prevents transactions in a disctributed database, and we will discuss implementations of distributed transactions in Chapter 9.

But do we need multi-object transactions at all? Would it be possible to implement any application with only a key-value data model and single-object operations?

There are some use cases in which single-object inserts, updates, and deletes are sufficient. However, in many other cases writes to several different objects need to be coordinated

#### Implementing read committed

Read committed is a very popular isolation level. It is the default setting in Oracle1 1g, PostgresSQL, SQL Server 2010, MemSQL, and many other databases.

Most commonly, databases prevent dirty writes by using row-level locks: when a transaaction wants to modify a particular object, it must first acquire a lock on that object. It must then hold that lock until the transaction is committed or aborted. Only one transaction can hold the lock for any given object; if another transaction wants to write to the same object, it must wait until the first transaction is committed or aborted before it can acquire the lock and continue. This locking is done automatically by databses in read committed mode.

How do we prevent dirty reads? One option would be to use the same lock, and to requre any transaction that wants to read an object to briefly acquire the lock and then releac it again immediately after reading. This would ensure that a read couldn't happen while an object has a dirty, uncommitted value.

However, the approach of requiring read locks does not work well in practice, because one long-running write transaction can force many read-only transactions to wait unltil the long-running transaction has completed. This harms the response time of read-only transactions and is bad for operability: a slowdown in one part of an application can have a knock-on effect in a completely different part of the application, due to waiting for locks.

- nonrepeatable read or read skew

- Snapshow isolation is the most common solution to this problem. The idea is that each transaction reads from a consistent snapshot of the database-that is, the transaction sees all the data that was committed in the database at the start of the transaction. Even if the data is subsequently changed by another transaction, each transaction sees only the old data from that particular point in time.

Snapshow isolation is a boon for long-running, read-only queries such as backups and analytics. It is very hard to reason about the meaning of a query if the data on which it operates is changing at the same time as the query is executing. Whne a transaction can see a consistent snapshow of the database, frozen at a particular point in time, it is much easier to understand.

- Multi-version concurrency control

### Visibility rules for observing a cinsistent snapshot

When a transaction reads from the database, transaction IDs are used to decide which objects it can see and which are invisible. By carefully defining visibility rules, the databse can present a consistent snapshot of the database to the application. This works as follows:

1. At the start of each transaction, the database makes a list of all the other transactions that are in progress at that time. Any writes that those transactions have made are ignored, even if the transactions subsequently commit.

#### Repeatalbe read and naming confusion

Snapshot isolation is a useful isolation levle, especially for read-only transactions. However, many databases that implement it call it by different names. In Oracle it is called serializable, and in PostgreSQL and MySQL it is called repeatable read.

The reason for this naming confusion is that the SQL standard doesn't have the concept of snapshow isolation, because the standard is based on System R's 1975 definition of isolation levels and snapshow isolation hadn't yet been invented then. Instead, it defines repeatable read, which looks superficially similar to snapshow isolation. PostgreSQL and MySQL call their snapshot isolation level repeatable read because it meets the requirements of the standard, and so they can claim standards compliance.

## Preventing Lost Updates

You can think of write skew as a generalization of the lost update problem. Wirte skew can occur if two transactions read the same objects, and then update some of those objects. In the special case where different transactions update the same object, you get a dirty write or lost update anomaly.

We saw that there are various different ways of preventing lost updates. With write skew, our options are more restricted:

### Phantoms causing write skew

1. A SELECT query checks whether some requirment is satisfied by searching for rows that match some search condition(there are at least tow doctors on call, there are no existing bookings for that room at that time, the position on the board doesn't already have another figure on it, the username isn't already takeb, there is still money in the account).
2. Depending on the result of the first query, the application code decides how to continue.
3. If the application decides to go ahead, it makes a wrtie to the database and commits the transaction.

#### Encapsulating transactions in stored procedures

In the early days of databases, the intention was that a database transaction could encompass an entire flow of user activity, For example, booking an airline ticket is a multi-stage process. Databses designers thought that it would be neat if that entire process was one transaction so that it could be committed atomically.

Unfortunatry, humans are very slow to make up their minds and respond. If a database transaction needs to wait for input from a user, the database needs to support a potentially huge number of concurrent transactions, most of them idle. Most databases cannot do that efficiently, and so almost all OLTP applications keep transactions short by avoiding interactively waiting for a user within a transation. On the web, this means that a transaction is committed within the same HTTP request-a transaction does not span multiple reqeusts. A new HTTP request starts a new transaction.